In [83]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show

from bokeh.models import HoverTool, WheelZoomTool, ResetTool, PanTool

#output_file("slider.html")
output_notebook()

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[ tab1, tab2 ])



show(tabs)

Loading BokehJS ...

In [52]:
import pandas as pd
import numpy as np

In [55]:
# pull in new data, extreme heat as a tester file

county_hex = '~/Code/jupyter-notebooks/cci-dv-2017/data/county-hex.csv'
cci =  pd.read_csv(county_hex)

# change county column name
cci.columns = ['county' if x=='County' else x for x in cci.columns]

# set index to counties
cci.set_index('county')

cci = cci.sort_values(by = "col-c", ascending=True)

cci = cci[['county','Population','row-c', 'col-c']]

# check out first lines
cci.head()


,county,Population,row-c,col-c
11,Humboldt,134876,2,1
7,Del Norte,28066,1,1
22,Mendocino,87612,3,1
19,Madera,152452,5,2
16,Lake,64209,4,2


In [60]:
# locations of Climate Change Indicator Data
extreme_heat = '~/Code/jupyter-notebooks/cci-dv-2017/data/extreme-heat.csv'
pm25 = '~/Code/jupyter-notebooks/cci-dv-2017/data/pm25.csv'
sea_level_rise ='~/Code/jupyter-notebooks/cci-dv-2017/data/sea-level-rise.csv'
ozone = '~/Code/jupyter-notebooks/cci-dv-2017/data/ozone.csv'
wildfire = '~/Code/jupyter-notebooks/cci-dv-2017/data/wildfire.csv'

red_values_dict = {0: '#ffffb2', 1: '#fecc5c', 2: '#fd8d3c', 3: '#f03b20', 4: '#bd0026'}


In [61]:
# read in extreme heat data set
eh = pd.read_csv(extreme_heat)
eh = eh[eh.ReportYear == 2050]
eh = eh.sort_values(by = "estimate", ascending=True)
#eh = eh.set_index('Geoname')
eh = eh[['County_Name','estimate']]
eh['eh_r'] = eh['estimate'].rank(ascending=1)
eh.columns = ['county' if x=='County_Name' else x for x in eh.columns]
eh.columns = ['eh_i' if x=='estimate' else x for x in eh.columns]
eh = eh.set_index('county')

# creating quintiles
eh['eh_q'] = pd.qcut(eh['eh_i'], 5, labels=False)
eh['eh_color'] = [red_values_dict.get(v, None) for v in eh['eh_q']]

eh.head()

,eh_i,eh_r,eh_q,eh_color
county,,,,
San Francisco,8.100000,1.0,0,#ffffb2
Santa Cruz,9.533333,2.0,0,#ffffb2
San Mateo,12.400000,3.0,0,#ffffb2
Marin,13.300000,4.0,0,#ffffb2
Orange,15.166667,5.0,0,#ffffb2


In [112]:
# Extreme Heat Bar
plot_options = dict(width=500, plot_height=250, tools='pan,wheel_zoom')

e = figure(**plot_options)

eh_i_mean = np.mean(eh['eh_i'], axis=0)

e.vbar(x=eh['eh_r'], width=0.5, bottom=0,
       top=eh['eh_i'], color=eh['eh_color'],)

e.line(x=[0,60], y=[eh_i_mean,eh_i_mean],line_width=3, line_color='black', line_alpha=0.5)

e.xaxis.visible = False
e.xgrid.visible = False
e.ygrid.visible = False

show(e)

In [93]:
wf =  pd.read_csv(wildfire,encoding ='latin1',low_memory=False)

# rename column names
wf.columns = ['county' if x=='County_Name' else x for x in wf.columns]
wf.columns = ['wf_i' if x=='estimate' else x for x in wf.columns]

# set index to counties
wf.set_index('county')

# grabbing only county geotype and all race
wf = wf[(wf.geotype == 'CO') & (wf.race_eth_name == 'Total')] 

#sort
wf = wf.sort_values(by = "wf_i", ascending=True)
wf['wf_r'] = wf['wf_i'].rank(ascending=1)

# quants
wf['wf_q'] = pd.qcut(wf['wf_i'], 5, labels=False)
wf['wf_color'] = [red_values_dict.get(v, None) for v in wf['wf_q']]

# just keep three bits
wf = wf[['county', 'wf_r', 'wf_i','wf_color']]
wf.head(10)

,county,wf_r,wf_i,wf_color
3869,San Francisco,2.5,0.000000e+00,#ffffb2
3671,Kings,2.5,0.000000e+00,#ffffb2
3986,Sutter,2.5,0.000000e+00,#ffffb2
3653,Inyo,2.5,0.000000e+00,#ffffb2
3743,Merced,5.0,2.000000e-07,#ffffb2
3878,San Joaquin,6.0,7.000000e-07,#ffffb2
3644,Imperial,7.0,5.760000e-05,#ffffb2
3977,Stanislaus,8.0,1.777000e-03,#ffffb2
3833,Sacramento,9.0,2.017410e-02,#ffffb2
3959,Solano,10.0,2.243200e-02,#ffffb2


In [109]:
# Wildfire Bar

hover_wf = HoverTool(tooltips=[
    ("County: ", "@county"),
    ("Wildfire: ", "@wf_i")],
     mode='mouse'
                    )

w = figure(plot_width=500, plot_height=250, tools=[WheelZoomTool(), PanTool(),ResetTool(),hover_wf])

wf_i_mean = np.mean(wf['wf_i'], axis=0)

w.vbar(x=range(len(wf['county'])), width=0.5, bottom=0,
       top='wf_i', color='wf_color', source=wf)

w.line(x=[0,60], y=[wf_i_mean,wf_i_mean],line_width=3, line_color='black', line_alpha=0.5)

w.xaxis.visible = False
w.xgrid.visible = False
w.ygrid.visible = False

show(w)

/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [110]:
output_file("tabby.html")

tab1 = Panel(child=w, title="wildfires")

tab2 = Panel(child=e, title="extreme heat")

tabs = Tabs(tabs=[ tab1, tab2 ])

show(tabs)